# Pandasとの比較

データ操作と分析の分野で広く利用されているライブラリとして、PolarsとPandasの二つがあります。これらのライブラリはどちらも強力なツールですが、その設計哲学やパフォーマンス特性にはいくつかの重要な違いがあります。以下では、PolarsとPandasの違いを比較し、Polarsの利点と欠点を明らかにします。

## 行のインデックス

DataFrameにおける行のインデックスの取り扱いに関して、PolarsとPandasには明確な違いがあります。

**Pandas:**
- **インデックス**: Pandasでは、DataFrameはインデックス（行ラベル）を持っています。デフォルトでは、インデックスは0から始まる整数ですが、カスタムのインデックスを指定することもできます。インデックスはデータ選択や操作を行う上で非常に便利です。
- **インデックスの役割**: インデックスを使用すると、特定の行を迅速にアクセスしたり、データをマージ、結合、グループ化したりすることが簡単になります。

In [1]:
import pandas as pd

data = {'name': ['Alice', 'Bob', 'Charlie'], 'age': [25, 30, 35], 'index':['a', 'b', 'c']}
df = pd.DataFrame(data).set_index('index')
df.loc[['a']] #この計算はO(1)

,name,age
index,,
a,Alice,25


**Polars:**
- **インデックスなし**: Polarsではインデックスの概念がありません。すべての操作は明示的に行われ、行ラベルを使用せずに行を選択します。
- **パフォーマンス**: インデックスを持たないため、Polarsはメモリ効率が高いです。

In [2]:
import polars as pl

df = pl.DataFrame(data)
df.filter(pl.col.index == 'a') #この計算はO(N)

name,age,index
str,i64,str
"""Alice""",25,"""a"""


## Polarsの列名は文字列のみ

**Pandas:**

Pandasでは、列名として文字列以外にも整数やタプルなど、さまざまな型を使用することができます。これにより、データフレームを柔軟に構造化できます。

In [16]:
data = {10: ['Alice', 'Bob', 'Charlie'], 20: [25, 30, 35]}
df = pd.DataFrame(data)
print(df[10])

0      Alice
1        Bob
2    Charlie
Name: 10, dtype: object


**Polars:**

Polarsでは、すべての列名は文字列でなければなりません。これは設計の簡素化と一貫性のためであり、列名が明確に定義されていることを保証します。すべての列名が文字列であるため、データ操作時に混乱が少なく、一貫したコードを書くことができます。

In [19]:
df = pl.DataFrame({str(key):value for key, value in data.items()})
print(df.select(pl.col('10')))

shape: (3, 1)
┌─────────┐
│ 10      │
│ ---     │
│ str     │
╞═════════╡
│ Alice   │
│ Bob     │
│ Charlie │
└─────────┘


## 演算式(Expression)

演算式は、データフレームの列に対する操作を表現します。Polarsでは演算式を使って、データ操作の一連の手続きを定義し、それを後で評価（実行）します。これにより、効率的かつ柔軟なデータ処理が可能になります。

**Pandas:**

まず、Pandasでの操作を見てみましょう。Pandasでは、操作はすぐに実行されます。次コードでは、条件に合った行を選択し、その場で給与を更新しています。


In [21]:
data = {'name': ['Alice', 'Bob', 'Charlie'], 'age': [25, 30, 35], 'salary': [50000, 60000, 70000]}
df = pd.DataFrame(data)

# 年齢が30以上の人を選択し、給与を5000増加させる
df.loc[df['age'] >= 30, 'salary'] += 5000
print(df)

      name  age  salary
0    Alice   25   50000
1      Bob   30   65000
2  Charlie   35   75000


**Polars:**

同じ操作をPolarsのエクスプレッションを使って行います。Polarsでは、データフレームの列を修正する際に「インプレース（inplace）」で変更するのではなく、新しい列を作成するという哲学を持っています。これは、データフレームが不変（immutable）であることを意味し、元のデータフレームは変更されず、新しいデータフレームや列が生成されます。

In [25]:
df = pl.DataFrame(data)

# 年齢が30以上の人を選択し、給与を5000増加させるエクスプレッションを定義
expr = pl.when(pl.col('age') >= 30).then(pl.col('salary') + 5000).otherwise(pl.col('salary'))

# エクスプレッションを適用して新しいデータフレームを作成
df = df.with_columns(expr.alias('updated_salary'))
print(df)

shape: (3, 4)
┌─────────┬─────┬────────┬────────────────┐
│ name    ┆ age ┆ salary ┆ updated_salary │
│ ---     ┆ --- ┆ ---    ┆ ---            │
│ str     ┆ i64 ┆ i64    ┆ i64            │
╞═════════╪═════╪════════╪════════════════╡
│ Alice   ┆ 25  ┆ 50000  ┆ 50000          │
│ Bob     ┆ 30  ┆ 60000  ┆ 65000          │
│ Charlie ┆ 35  ┆ 70000  ┆ 75000          │
└─────────┴─────┴────────┴────────────────┘


このコードでは、`expr`というエクスプレッションを定義し、それをデータフレームに適用しています。このエクスプレッションは「年齢が30以上なら給与を5000増やし、それ以外なら元の給与を保持する」という操作を表現しています。

### 演算式の利点

1. **遅延評価**: Polarsのエクスプレッションは定義時に実行されず、明示的に評価（実行）されるまで待機します。これにより、不要な計算を避け、効率的にデータ処理を行えます。
2. **チェーン操作**: 複数のエクスプレッションをチェーンして、複雑なデータ操作を簡潔に記述できます。
3. **パフォーマンス**: Polarsはエクスプレッションを最適化して一度に実行するため、大規模データセットでも高いパフォーマンスを発揮します。


### 非インプレース操作の理由と利点

1. 安全性: データフレームを変更しないことで、元のデータが保護されます。これにより、意図しない変更やバグを防ぐことができます。
2. デバッグの容易さ: 元のデータフレームはそのまま残るため、データの変化を追跡しやすく、デバッグが容易になります。
3. チェーン操作のサポート: 各操作が新しいデータフレームを返すため、メソッドチェーンを使った直感的なデータ操作が可能です。

## まとめ

- **インデックス**: Pandasはインデックスを持ち、データ操作の柔軟性が高いが、Polarsはインデックスを持たず、シンプルで高速な操作が可能。
- **列名**: Pandasは多様な列名をサポートし柔軟性が高いが、Polarsは列名を文字列に限定し一貫性を保つ。
- **エクスプレッション**: Polarsの強力な機能であり、遅延評価とチェーン操作を活用することで、効率的かつ柔軟なデータ操作が可能になります。Pandasにはこのような中間表現の概念がないため、Polarsのエクスプレッションは特に大規模データセットの処理や複雑なデータ操作において有用です。

PolarsとPandasのそれぞれの特性を理解することで、目的に応じて最適なライブラリを選択することができます。